In [ ]:
# Code to reassign the player the same ID even after he moves out of the frame and comes back.
from tracker import Player, iou, compare_players

from ultralytics import YOLO
from torchvision import models, transforms
import torch
import torch.nn as nn
import numpy as np
import cv2
import os
import json
from PIL import Image


In [ ]:
# Pretrained model for detection
model = YOLO("best.pt")

# ResNet50 model for feature extraction
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove classification layer
resnet.eval()

# Image transform for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225])
])



In [ ]:
def cosine_similarity(vec1, vec2):
    vec1 = vec1 / np.linalg.norm(vec1)
    vec2 = vec2 / np.linalg.norm(vec2)
    return np.dot(vec1, vec2)



In [ ]:
VIDEO_PATH = "15sec_input_720p.mp4"
OUTPUT_DIR = "detected_frames_day3"
CROPS_DIR = "players_crop_day3"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CROPS_DIR, exist_ok=True)


In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
frame_id = 0
player_id_counter = 0
active_players = []
tracking_output = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = []

    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            if cls != 0 and cls != 24:  # Only detect persons and players
                continue
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            crop = frame[y1:y2, x1:x2]
            detections.append({
                "bbox": (x1, y1, x2, y2),
                "conf": conf,
                "crop": crop,
                "assigned": False
            })

    for det in detections:
        matched = False
        try:
            pil_crop = Image.fromarray(det["crop"])
            input_tensor = transform(pil_crop).unsqueeze(0)
            with torch.no_grad():
                feat = resnet(input_tensor).squeeze().numpy()
        except:
            continue

        best_score = 0
        best_player = None

        for player in active_players:
            if player.feature is None or frame_id - player.last_seen > 50:
                continue
            score = cosine_similarity(feat, player.feature)
            if score > best_score:
                best_score = score
                best_player = player

        if best_score > 0.75:
            best_player.update(det["bbox"], frame_id, det["crop"], feat)
            det["assigned"] = True
            tracking_output.append({
                "frame": frame_id,
                "id": best_player.id,
                "bbox": det["bbox"]
            })

            # ✅ Save crop even for matched players
            crop_path = f"{CROPS_DIR}/frame_{frame_id:03}_id_{best_player.id}.jpg"
            if det["crop"].shape[0] > 50 and det["crop"].shape[1] > 30:
                cv2.imwrite(crop_path, det["crop"])

        else:
            new_player = Player(player_id_counter, det["bbox"], frame_id, det["crop"], feat)
            active_players.append(new_player)
            tracking_output.append({
                "frame": frame_id,
                "id": player_id_counter,
                "bbox": det["bbox"]
            })

            # ✅ Save crop for new players
            crop_path = f"{CROPS_DIR}/frame_{frame_id:03}_id_{player_id_counter}.jpg"
            if det["crop"].shape[0] > 50 and det["crop"].shape[1] > 30:
                cv2.imwrite(crop_path, det["crop"])

            player_id_counter += 1

    for det in tracking_output:
        if det["frame"] == frame_id:
            x1, y1, x2, y2 = det["bbox"]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {det['id']}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out_path = os.path.join(OUTPUT_DIR, f"frame_{frame_id:03}.jpg")
    cv2.imwrite(out_path, frame)
    frame_id += 1

cap.release()


In [ ]:
with open("tracking_output.json", "w") as f:
    json.dump(tracking_output, f, indent=4)


In [ ]:
from IPython.display import Image, display
for i in range(5):
    frame_path = os.path.join(OUTPUT_DIR, f"frame_{i:03}.jpg")
    if os.path.exists(frame_path):
        display(Image(filename=frame_path))

